# 📊 PDF 데이터시트 전처리 도구

**데이터시트 PDF를 분석하여 유용한 정보를 추출하고 JSON 형태로 변환합니다.**

---

## 🚀 시작하기 전에

이 노트북은 PDF 데이터시트를 처리하여 나중에 챗봇에서 사용할 수 있는 JSON 파일을 생성합니다.

## 📋 사용 방법
아래 셀들을 **순서대로 실행**만 하면 됩니다. (각 셀의 ▶️ 버튼을 클릭)

---

In [ ]:
#@title ✅ 1단계: Google Drive 연동 { display-mode: "form" }
#@markdown Google Drive를 연동하여 PDF 파일을 불러오고 처리된 JSON 파일을 저장합니다.

from google.colab import drive
import os
from pathlib import Path

print("🔄 Google Drive 마운트 중...")
drive.mount('/content/drive')

# PDF와 JSON을 위한 폴더 생성
pdf_folder = Path('/content/drive/MyDrive/datasheets')
json_folder = Path('/content/drive/MyDrive/processed_json')

# 폴더가 없으면 생성
pdf_folder.mkdir(parents=True, exist_ok=True)
json_folder.mkdir(parents=True, exist_ok=True)

print(f"\n✅ Google Drive 연동 완료!")
print(f"📁 PDF 폴더: {pdf_folder}")
print(f"📁 JSON 폴더: {json_folder}")

# PDF 파일 목록 확인
pdf_files = list(pdf_folder.glob("*.pdf"))
print(f"\n📄 발견된 PDF 파일: {len(pdf_files)}개")
for pdf in pdf_files[:5]:  # 처음 5개만 표시
    print(f" - {pdf.name}")
    
if len(pdf_files) > 5:
    print(f" ... 외 {len(pdf_files) - 5}개")

if not pdf_files:
    print("\n⚠️ PDF 파일이 없습니다. '{pdf_folder}' 폴더에 PDF 파일을 업로드해주세요.")

In [ ]:
#@title 📥 2단계: 필요한 라이브러리 설치 { display-mode: "form" }
#@markdown 필요한 라이브러리들을 설치합니다.

import os
from pathlib import Path

print("📥 필요한 라이브러리 설치 중...")

!pip install -r requirements.txt -q

# 현재 디렉토리 설정
import sys
current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))
sys.path.append(str(current_dir / 'src'))

print("✅ 라이브러리 설치 완료!")

In [ ]:
#@title 🔑 3단계: OpenAI API 키 설정 { display-mode: "form" }
#@markdown OpenAI API 키를 입력하여 PDF 분석을 위한 LLM을 설정합니다.

import os

api_key = "" #@param {type:"string"}

if not api_key:
    print("⚠️ API 키가 입력되지 않았습니다.")
    print("💡 OpenAI API 키를 입력해주세요. (https://platform.openai.com/api-keys)")
else:
    os.environ["OPENAI_API_KEY"] = api_key
    print("✅ API 키 설정 완료!")

In [ ]:
#@title 📊 4단계: PDF 파일 처리 { display-mode: "form" }
#@markdown 선택한 PDF 파일을 처리합니다.

from src.pdf_processor import PDFProcessor
import json
import os

# API 키 확인
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    print("❌ API 키가 설정되지 않았습니다. 3단계를 먼저 완료해주세요.")
else:
    # PDF 처리기 초기화
    processor = PDFProcessor(os.environ["OPENAI_API_KEY"])
    
    # 처리할 PDF 파일 선택
    if not pdf_files:
        print("❌ 처리할 PDF 파일이 없습니다. Google Drive에 파일을 업로드해주세요.")
    else:
        print("📝 처리할 PDF 파일 목록:")
        for i, pdf in enumerate(pdf_files):
            print(f"{i+1}. {pdf.name}")
            
        # PDF 인덱스 입력 (1부터 시작)
        pdf_index = 1  #@param {type:"integer"}
        
        if pdf_index < 1 or pdf_index > len(pdf_files):
            print(f"❌ 잘못된 인덱스입니다. 1에서 {len(pdf_files)} 사이의 값을 입력하세요.")
        else:
            # 선택한 PDF 처리
            selected_pdf = pdf_files[pdf_index-1]
            print(f"\n🔍 '{selected_pdf.name}' 파일을 처리합니다...")
            print("⏳ 이 작업은 몇 분 정도 걸릴 수 있습니다. 잠시만 기다려주세요...")
            
            try:
                features, useful_pages = processor.process_pdf(str(selected_pdf))
                
                # 결과 저장
                if features:
                    output_file = json_folder / f"{selected_pdf.stem}.json"
                    with open(output_file, 'w', encoding='utf-8') as f:
                        json.dump(features, f, indent=2, ensure_ascii=False)
                        
                    print(f"\n✅ 처리 완료!")
                    print(f"📄 추출된 유용한 페이지: {useful_pages}")
                    print(f"💾 결과가 저장되었습니다: {output_file}")
                    print("\n📊 추출된 정보 미리보기:")
                    print(json.dumps(features, indent=2, ensure_ascii=False)[:500] + '...')
                else:
                    print("❌ 유용한 정보를 추출하지 못했습니다.")
            except Exception as e:
                print(f"❌ 처리 중 오류가 발생했습니다: {str(e)}")

In [ ]:
#@title 🔄 5단계: 모든 PDF 파일 일괄 처리 { display-mode: "form" }
#@markdown 모든 PDF 파일을 한 번에 처리합니다. (시간이 오래 걸릴 수 있습니다)

import os
import json
from tqdm.notebook import tqdm

# API 키 확인
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    print("❌ API 키가 설정되지 않았습니다. 3단계를 먼저 완료해주세요.")
else:
    # PDF 처리기 초기화
    processor = PDFProcessor(os.environ["OPENAI_API_KEY"])
    
    # 처리할 PDF 파일 확인
    if not pdf_files:
        print("❌ 처리할 PDF 파일이 없습니다. Google Drive에 파일을 업로드해주세요.")
    else:
        # 일괄 처리 여부 확인
        confirm = True  #@param {type:"boolean"}
        
        if confirm:
            print(f"🔄 {len(pdf_files)}개의 PDF 파일을 처리합니다...")
            print("⏳ 이 작업은 시간이 오래 걸릴 수 있습니다. 잠시만 기다려주세요...")
            
            # 처리 결과 저장
            results = {}
            
            # tqdm으로 진행 상황 표시
            for pdf in tqdm(pdf_files, desc="PDF 처리 중"):
                try:
                    features, useful_pages = processor.process_pdf(str(pdf))
                    
                    if features:
                        # 결과 저장
                        output_file = json_folder / f"{pdf.stem}.json"
                        with open(output_file, 'w', encoding='utf-8') as f:
                            json.dump(features, f, indent=2, ensure_ascii=False)
                        
                        results[pdf.name] = {
                            "status": "성공",
                            "useful_pages": useful_pages,
                            "output_file": str(output_file)
                        }
                    else:
                        results[pdf.name] = {
                            "status": "실패",
                            "reason": "유용한 정보를 추출하지 못했습니다."
                        }
                except Exception as e:
                    results[pdf.name] = {
                        "status": "오류",
                        "reason": str(e)
                    }
            
            # 결과 요약
            success_count = sum(1 for result in results.values() if result["status"] == "성공")
            failed_count = len(results) - success_count
            
            print(f"\n✅ 처리 완료! 총 {len(results)}개 중 {success_count}개 성공, {failed_count}개 실패")
            print(f"💾 결과가 '{json_folder}'에 저장되었습니다.")
            
            if failed_count > 0:
                print("\n⚠️ 실패한 파일:")
                for name, result in results.items():
                    if result["status"] != "성공":
                        print(f" - {name}: {result.get('reason', '알 수 없는 오류')}")
        else:
            print("❌ 일괄 처리가 취소되었습니다.")

## 🎉 처리 완료!

모든 PDF 파일이 처리되었습니다. 생성된 JSON 파일은 Google Drive의 `processed_json` 폴더에 저장되었습니다.

이제 ChipChat 모듈에서 이 JSON 파일들을 불러와 데이터시트에 대한 질의응답을 수행할 수 있습니다.